# Getting information from SBML PBK model files

This notebook demonstrates how information can be obtained from a PBK model stored in the SBML format using the EuroMix PBK model SBML model as example.

## Preparation

In [25]:
import pandas as pd
import libsbml as ls
from tabulate import tabulate

In [26]:
# Specify sbml file path
sbml_file = '../model/euromix.sbml'

## Parsing the model

### General SBML file format checks

In [27]:
# Open SBML document using libSBML
document = ls.readSBML(sbml_file)

# Check for errors
if document.getNumErrors() > 0:
    print("Encountered " + str(document.getNumErrors()) + " SBML errors:" )
    document.printErrors()

# Get SBML level and version and print
level = document.getLevel()
version = document.getVersion()
print("\n"
      + "File: " + sbml_file
      + " (Level " + str(level) + ", version " + str(version) + ")" )

# Get the model
model = document.getModel()

if model is None:
    print("No model present." )


File: ../model/euromix.sbml (Level 3, version 2)


### Describe compartments

Each compartment should be linked to a compartment matching the vocabulary of the PBK modelling ontology. For each compartment, the unit of measurement of the volume should be specified.

In [28]:
# Helper function to extract is-a resource URI
def getTerm(element):
    cvTerms = element.getCVTerms()
    if not cvTerms:
        return None
    for term in cvTerms:
      num_resources = term.getNumResources()
      for j in range(num_resources):
          if term.getQualifierType() == ls.MODEL_QUALIFIER and \
              term.getModelQualifierType() == ls.BQM_IS:
              return term.getResourceURI(j)
          elif term.getQualifierType() == ls.BIOLOGICAL_QUALIFIER and \
              term.getBiologicalQualifierType() == ls.BQB_IS:
              return term.getResourceURI(j)
    return None 

# Create compartments info table
print("Compartments (" + str(model.getNumCompartments()) + ")")
dt_compartments = []
for i in range(0,model.getNumCompartments()):
    c = model.getCompartment(i)
    dt_compartments.append([
        c.getId(),
        c.getName(),
        c.getVolume(),
        c.getUnits(),
        getTerm(c)
    ])
df_compartments = pd.DataFrame(
    dt_compartments,
    columns=["Id", "Name", "Volume", "Unit", "URI"]
)
print(tabulate(df_compartments, headers="keys", tablefmt="fancy_grid", showindex=False))

Compartments (13)
╒═══════════╤═════════════════════════════════╤══════════╤════════╤════════════════════════════════════════════╕
│ Id        │ Name                            │   Volume │ Unit   │ URI                                        │
╞═══════════╪═════════════════════════════════╪══════════╪════════╪════════════════════════════════════════════╡
│ Air       │ alveolar air                    │  1       │ L      │ http://purl.obolibrary.org/obo/PBPKO_00448 │
├───────────┼─────────────────────────────────┼──────────┼────────┼────────────────────────────────────────────┤
│ Urine     │ urine                           │  1       │ L      │ http://purl.obolibrary.org/obo/PBPKO_00556 │
├───────────┼─────────────────────────────────┼──────────┼────────┼────────────────────────────────────────────┤
│ Fat       │ adipose tissue                  │ 14.63    │ L      │ http://purl.obolibrary.org/obo/PBPKO_00460 │
├───────────┼─────────────────────────────────┼──────────┼────────┼───────────

### Describe substances (chemical species)

Each species should represent a substance concentration/amount of a chemical substance in one of the compartments. For each species, the unit of measurement of the concentration/amount should be specified.

In [29]:
# Create species info table
dt_species = []
print("Number of species: " + str(model.getNumSpecies()))
for i in range(0,model.getNumSpecies()):
    s = model.getSpecies(i)
    dt_species.append([
        s.getId(),
        s.getName(),
        s.getCompartment(),
        s.getUnits()
    ])
df_species = pd.DataFrame(
    dt_species,
    columns=["Id", "Name", "IdCompartment", "Unit"]
)
print(tabulate(df_species, headers="keys", tablefmt="fancy_grid", showindex=False))

Number of species: 14
╒════════════╤══════════════════════════════════════════════════════════════╤═════════════════╤══════════╕
│ Id         │ Name                                                         │ IdCompartment   │ Unit     │
╞════════════╪══════════════════════════════════════════════════════════════╪═════════════════╪══════════╡
│ QFat       │ amount of chemical in fat tissues                            │ Fat             │ MilliMOL │
├────────────┼──────────────────────────────────────────────────────────────┼─────────────────┼──────────┤
│ QRich      │ amount of chemical in richly tissues                         │ Rich            │ MilliMOL │
├────────────┼──────────────────────────────────────────────────────────────┼─────────────────┼──────────┤
│ QPoor      │ amount of chemical in poor tissues                           │ Poor            │ MilliMOL │
├────────────┼──────────────────────────────────────────────────────────────┼─────────────────┼──────────┤
│ QLiver     │ 

### Describe parameters

We distinguish between internal and external parameters. Internal parameters are set by the model, via initial assignment rules, based on other parameters. External parameters are parameters that can be specified in model parametrisations. In SBML, these the *isSetConstant* method can be used to distinguish between these types of parameters.

The unit for each parameter should be specified. Unitless parameters should be explicitly specified as such. Each parameter should be associated with the correct ontology term(s) that exactly identify the type of parameter so that it becomes possible to search for the parameter value in databases.

In [30]:
# Create parameters info table
dt_params = []
print("Number of parameters: " + str(model.getNumParameters()))
for i in range(0,model.getNumParameters()):
    s = model.getParameter(i)
    dt_params.append([
        s.getId(),
        s.getName(),
        s.getUnits(),
        s.getConstant(),
        getTerm(s)
    ])
df_params = pd.DataFrame(
    dt_params,
    columns=["Id", "Name", "Unit", "IsExternal", "URI"]
)
print(tabulate(df_params, headers="keys", tablefmt="fancy_grid", showindex=False))

Number of parameters: 42
╒═════════════╤═════════════════════════════════════════════════════════════════╤═══════════════════╤══════════════╤════════════════════════════════════════════╕
│ Id          │ Name                                                            │ Unit              │ IsExternal   │ URI                                        │
╞═════════════╪═════════════════════════════════════════════════════════════════╪═══════════════════╪══════════════╪════════════════════════════════════════════╡
│ BM          │ body weight                                                     │ KiloGM            │ True         │ http://purl.obolibrary.org/obo/PBPKO_00008 │
├─────────────┼─────────────────────────────────────────────────────────────────┼───────────────────┼──────────────┼────────────────────────────────────────────┤
│ BSA         │ body surface area                                               │ DeciM2            │ True         │ http://purl.obolibrary.org/obo/PBPKO_00010 │
├──

## Describe units

These are the units defined in the model.

In [31]:
dt_params = []
print("Number of units: " + str(model.getNumUnitDefinitions()))
for i in range(0,model.getNumUnitDefinitions()):
    s = model.getUnitDefinition(i)
    dt_params.append([
        s.getId(),
        ls.UnitDefinition.printUnits(s)
    ])
df_params = pd.DataFrame(
    dt_params,
    columns=["Id", "Unit"]
)
print(tabulate(df_params, headers="keys", tablefmt="fancy_grid", showindex=False))

Number of units: 13
╒═══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ Id                │ Unit                                                                                                                                                    │
╞═══════════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ MilliMOL          │ mole (exponent = 1, multiplier = 1, scale = -3)                                                                                                         │
├───────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ HR                │ second (exponent = 1, multiplier = 3600, scale = 0)                           

### Render the differential equations

Script below renders the differential equations of the model in simple text format.

In [32]:
# Get the list of species IDs
species_ids = [species.getId() for species in model.getListOfSpecies()]

# Initialize dictionaries to store the differential equations and reaction rates
differential_equations = {}
reaction_rates = {}

# Iterate over the reactions in the model
for i in range(model.getNumReactions()):
    reaction = model.getReaction(i)
    
    # Get the kinetic law associated with the reaction
    kinetic_law = reaction.getKineticLaw()
    if kinetic_law is None:
        continue
    
    # Get the reaction rate formula
    rate_formula = ls.formulaToString(kinetic_law.getMath())
    
    # Iterate over the reactants and products of the reaction
    for j in range(reaction.getNumReactants()):
        reactant = reaction.getReactant(j)
        species_id = reactant.getSpecies()
        
        # Add the reaction rate to the dictionary
        if species_id in reaction_rates:
            reaction_rates[species_id].append(f"- {rate_formula}")
        else:
            reaction_rates[species_id] = [f"- {rate_formula}"]
    
    for j in range(reaction.getNumProducts()):
        product = reaction.getProduct(j)
        species_id = product.getSpecies()
        
        # Add the negative reaction rate to the dictionary
        if species_id in reaction_rates:
            reaction_rates[species_id].append(rate_formula)
        else:
            reaction_rates[species_id] = [rate_formula]

# Construct the differential equations for each species
for species_id in species_ids:
    if species_id in reaction_rates:
        new_line = '\n'
        pref = f'{new_line}{" " * (len(species_id) + 8)}'
        equation = f"d[{species_id}]/dt = {f'{pref} + '.join(reaction_rates[species_id])}"
        differential_equations[species_id] = equation.replace("+ -", "-")

# Print the differential equations for each species
for species_id, equation in differential_equations.items():
    print(equation)

d[QFat]/dt = FFat * (QArt / Art)
             - FFat * (QFat / Fat) / PCFat
d[QRich]/dt = FRich * (QArt / Art)
              - FRich * (QRich / Rich) / PCRich
d[QPoor]/dt = FPoor * (QArt / Art)
              - FPoor * (QPoor / Poor) / PCPoor
d[QLiver]/dt = FLiver * (QArt / Art)
               - FLiver * (QLiver / Liver) / PCLiver
               + kGut * QGut
               - fub * piecewise(metab_MM(Vmax, Km, PCLiver, QLiver, Liver), gt(Michaelis, 0.5), metab_MA(CLH, PCLiver, QLiver, Liver))
d[QMetab]/dt = fub * piecewise(metab_MM(Vmax, Km, PCLiver, QLiver, Liver), gt(Michaelis, 0.5), metab_MA(CLH, PCLiver, QLiver, Liver))
d[QGut]/dt = - kGut * QGut
d[QSkin_u]/dt = FSkin_u * (QArt / Art)
                - FSkin_u * conc(QSkin_u, Skin_u) / PCSkin
                - f_su * conc(QSkin_u, Skin_u) / PCSkin_sc
                + f_su * conc(QSkin_sc_u, Skin_sc_u)
d[QSkin_e]/dt = FSkin_e * (QArt / Art)
                - FSkin_e * conc(QSkin_e, Skin_e) / PCSkin
                - f_se * conc(QSki